In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
from fastai.vision.all import *


In [ ]:
labels = pd.read_csv("../input/dog-breed-identification/labels.csv")
labels


In [ ]:
labels["fname"] = labels["id"].apply(lambda x: x + ".jpg")
path = "../input/dog-breed-identification/train"

dls = ImageDataLoaders.from_df(labels, path, fn_col=2,
                               item_tfms=Resize(224),
                               batch_tfms=Normalize)


In [ ]:
dls.show_batch()


In [ ]:
def log_loss(inputs, targ):
    preds = torch.softmax(inputs, dim=1)
    return -1.0 * preds.gather(1, targ.view(-1, 1)).log().mean()


In [ ]:
learn = cnn_learner(dls, resnet50, loss_func=log_loss, path=".").to_fp16()


In [ ]:
learn.lr_find()


In [ ]:
learn.fine_tune(5, 3e-3)


In [ ]:
dls2 = ImageDataLoaders.from_df(labels, path, fn_col=2,
                               item_tfms=Resize(320),
                               batch_tfms=[*aug_transforms(), Normalize])
learn.dls = dls2


In [ ]:
learn.freeze()
learn.lr_find()


In [ ]:
learn.fine_tune(5, 1e-4)


In [ ]:
test_files = get_image_files("../input/dog-breed-identification/test")
test_dl = dls2.test_dl(test_files)


In [ ]:
preds, targs = learn.get_preds(dl=test_dl)


In [ ]:
preds = torch.softmax(preds, dim=1)
sub = pd.DataFrame({"id":test_files.map(lambda x:x.stem)})
sub[list(dls.vocab)] = preds
sub.to_csv("submission.csv", index=False)


In [ ]:
sub
